In [1]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')
mnist

C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)


{'COL_NAMES': ['label', 'data'],
 'DESCR': 'mldata.org dataset: mnist-original',
 'data': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'target': array([0., 0., 0., ..., 9., 9., 9.])}

In [2]:
X, y = mnist['data'] , mnist['target']

In [3]:
X.shape

(70000, 784)

In [4]:
y.shape

(70000,)

In [5]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

shuffled_index = np.random.permutation(len(X))
X = X[shuffled_index]
y = y[shuffled_index]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=10000, random_state=42)

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=10000, random_state=42)

In [7]:
X_train.shape

(50000, 784)

In [8]:
X_valid.shape

(10000, 784)

In [9]:
X_test.shape

(10000, 784)

RandomForestClassifier

In [10]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16)
rnd_clf.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=16,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

ExtraTreesClassifier

In [11]:
from sklearn.ensemble import ExtraTreesClassifier 

ext_clf = ExtraTreesClassifier(n_estimators=500, max_leaf_nodes=16)
ext_clf.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=16,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

LinearSVC 

In [12]:
from sklearn.svm import LinearSVC

lin_svc = LinearSVC(loss='hinge',C=1)
lin_svc.fit(X_train, y_train)

C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0)

In [13]:
rnd_y = rnd_clf.predict(X_valid)
ext_y = ext_clf.predict(X_valid)
lin_y = lin_svc.predict(X_valid)

In [14]:
from sklearn.metrics import accuracy_score

print('RandomForest:',accuracy_score(rnd_y, y_valid))
print('ExtraTree:',accuracy_score(ext_y, y_valid))
print('LinearSVC:',accuracy_score(lin_y, y_valid))

RandomForest: 0.8252
ExtraTree: 0.8144
LinearSVC: 0.8551


Voting Classifier

hard voting 

In [35]:
from sklearn.ensemble import VotingClassifier

hard_clf = VotingClassifier(estimators=[('rnd',rnd_clf),('ext',ext_clf),('linsvc',lin_svc)],
                           voting='hard')
hard_clf.fit(X_train, y_train)

C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


VotingClassifier(estimators=[('rnd', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=16,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_wei...e', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None)

In [36]:
hard_clf.estimators

[('rnd',
  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
              max_depth=None, max_features='auto', max_leaf_nodes=16,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
              oob_score=False, random_state=None, verbose=0,
              warm_start=False)),
 ('ext',
  ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=16,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
             oob_score=False, random_state=None, verbose=0, warm_start=False)),
 ('linsvc', LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
       intercept_scaling=1, loss='hinge', max_i

In [37]:
[clf.score(X_valid, y_valid) for clf in hard_clf.estimators_]

[0.8254, 0.8148, 0.8685]

In [38]:
hard_clf.score(X_valid, y_valid)

0.8371

In [39]:
hard_clf.set_params(ext=None)

VotingClassifier(estimators=[('rnd', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=16,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_wei...e', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None)

In [40]:
hard_clf.estimators

[('rnd',
  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
              max_depth=None, max_features='auto', max_leaf_nodes=16,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
              oob_score=False, random_state=None, verbose=0,
              warm_start=False)),
 ('ext', None),
 ('linsvc', LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
       intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
       penalty='l2', random_state=None, tol=0.0001, verbose=0))]

In [41]:
hard_clf.estimators_

[RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=16,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
 ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=16,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
            oob_score=False, random_state=None, verbose=0, warm_start=False),
 LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
      penalty='l

In [42]:
del hard_clf.estimators_[1]

In [43]:
hard_clf.score(X_valid, y_valid)

0.8359

soft voting

In [44]:
from sklearn.ensemble import VotingClassifier

soft_clf = VotingClassifier(estimators=[('rnd',rnd_clf),('ext',ext_clf)],
                           voting='soft')
soft_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rnd', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=16,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_wei...tors=500, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False))],
         flatten_transform=None, n_jobs=None, voting='soft', weights=None)

In [45]:
[clf.score(X_valid, y_valid) for clf in soft_clf.estimators_]

[0.826, 0.815]

In [46]:
soft_clf.score(X_valid, y_valid)

0.8241

stacking ensemble

In [20]:
estimators = [rnd_clf, ext_clf, lin_svc]

X_valid_pred = np.zeros([len(X_valid),len(estimators)])

for index, estimator  in enumerate(estimators) :
    X_valid_pred[:,index] = estimator.predict(X_valid) 

In [21]:
X_valid_pred

array([[1., 1., 1.],
       [7., 7., 7.],
       [4., 4., 4.],
       ...,
       [3., 3., 3.],
       [1., 1., 9.],
       [0., 0., 0.]])

In [22]:
rnd_blender =  RandomForestClassifier(n_estimators = 500, oob_score = True, random_state=42)
rnd_blender.fit(X_valid_pred, y_valid)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
            oob_score=True, random_state=42, verbose=0, warm_start=False)

In [24]:
rnd_blender.oob_score_

0.878

test

In [26]:
X_test_pred = np.zeros([len(X_test),len(estimators)])

for index, estimator  in enumerate(estimators) :
    X_test_pred[:,index] = estimator.predict(X_test) 

In [27]:
pred_y = rnd_blender.predict(X_test_pred)
accuracy_score(pred_y, y_test)

0.8819

XGboost Classifier

In [29]:
try:
    import xgboost
except ImportError as ex:
    print("Error: the xgboost library is not installed.")
    xgboost = None

In [33]:
param_dist = {'objective':'binary:logistic', 'n_estimators':3}
xgb_clf = xgboost.XGBClassifier(**param_dist)

xgb_clf.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_metric='mlogloss',
        verbose=True)

evals_result = xgb_clf.evals_result()

[0]	validation_0-mlogloss:2.08144	validation_1-mlogloss:2.08108
[1]	validation_0-mlogloss:1.90688	validation_1-mlogloss:1.90713
[2]	validation_0-mlogloss:1.76655	validation_1-mlogloss:1.76946


In [34]:
pred_y = xgb_clf.predict(X_valid)
accuracy_score(pred_y, y_valid)

0.7832